# VisDrone 2025 object Detection for KAU - Using YOLO11x

## Setup & imports

In [ ]:
%pip install ultralytics -q 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import PIL
import supervision as sv

In [ ]:
from ultralytics import YOLO
import ultralytics
ultralytics.checks()

In [ ]:
yaml_text="""
path: /kaggle/input/visdrone-kau-2025-dataset-yolo-format
train: train/images
val: val/images
test: test/images

names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor
"""
with open("/kaggle/working/visdrone.yaml", 'w') as f:
    f.write(yaml_text)
yaml = '/kaggle/working/visdrone.yaml'

## Training

We fine-tuned Yolo11x model for 100 epohs, with the next parameters:
* *batch size* - **24**
* *optimizer* - **Adam**
* *learning rate* - **0.001**
* *image size* - **640**

For training, we used **2x Tesla T4** GPU on Kaggle

In [ ]:
model = YOLO("yolo11x.pt")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/kaggle/working/runs/"

In [ ]:
results = model.train(data=yaml, epochs=100, batch=24, lr0=0.001, imgsz=640, device=[0,1], optimizer="Adam", save_period=10, cache='ram', pretrained=True)

## Training results

In [ ]:
 %matplotlib inline

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/results.png")

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/confusion_matrix.png")

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/val_batch1_pred.jpg")

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/F1_curve.png")

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/P_curve.png")

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/R_curve.png")

In [ ]:
PIL.Image.open("/kaggle/working/runs/detect/train4/PR_curve.png")

### Model Score

In [ ]:
model = YOLO("/kaggle/working/runs/detect/train4/weights/best.pt")

In [ ]:
model.val();

## Inference results
### Labels

In [ ]:
res = model("/kaggle/input/visdrone-kau-2025-dataset-yolo-format/val/images/0000001_02999_d_0000005.jpg")[0]
res.show()

In [ ]:
res = model("/kaggle/input/visdrone-kau-2025-dataset-yolo-format/val/images/0000001_05999_d_0000011.jpg")[0]
res.show()

## Submission generation

In [ ]:
model = YOLO("/kaggle/working/runs/detect/train4/weights/best.pt")

In [ ]:
test_results = model("/kaggle/input/visdrone-kau-2025-dataset-yolo-format/test/images/")

In [ ]:
from tqdm.notebook import tqdm
lines = []
for result in tqdm(test_results):
    index = int(result.path.split('.')[-2].split('/')[-1])
    confs = result.boxes.conf
    cls = result.boxes.cls
    boxes = result.boxes.xyxy
    line = f"{index},"
    sublines = []
    for lb, scr, box in zip(cls, confs, boxes):
        l = f"{int(lb)} {(float(scr))} "
        l+=' '.join(map(str, box.cpu().numpy().astype(int).tolist()))
        sublines.append(l)
    line += " | ".join(sublines)
    
    if line== f"{index},":
        line+=" | ".join(prev)
    else:
        prev = sublines
    line+='\n'
    
    lines.append(line)

In [ ]:
with open('submission-yolo11x-100.csv', 'w') as f:
    f.write("index,prediction_string\n")
    lines[-1] = lines[-1][:-1]
    f.writelines(lines)